In [12]:
#!pip install pandas
#!pip install yfinance
#!pip install matplotlib

import pandas as pd
import os
from datetime import datetime
import yfinance as yf
import Ab as ab
import datetime as dt

sd = dt.datetime(2000,1,1)
ed = dt.datetime(2023,11,17)

#read earning collection date from ./EarningCollectionDate.csv
earning_collection_date = pd.read_csv('./EarningCollectionDate.csv')
#column 'ticker', 'earning performance'
#convert 'ticker' to upper case
earning_collection_date['ticker'] = earning_collection_date['ticker'].str.upper()
#set the index to 'ticker'
earning_collection_date.set_index('ticker', inplace=True)
#if 'same' in 'earning performance' save 0 to 'performance date'
#if 'next' or 'next day' in 'earning performance' save 1 to 'performance date'
earning_collection_date['performance date'] = earning_collection_date['earning performance'].apply(lambda x: 0 if 'same' in x else 1)

#convert earning_collection_date to dictionary
earning_collection_date_dict = earning_collection_date.to_dict()['performance date']
print(earning_collection_date_dict)

# Define the directory path
directory = './Data_Feng/'
output = './Processed_Data_Feng/'

# Get a list of all the files in the directory
files = os.listdir(directory)
all_data = pd.DataFrame()
# Iterate over each file in the directory
for file in files:
    stock_name = file.split('.')[0]

    #convert stock_name to upper case
    stock_name = stock_name.upper()
    print(stock_name)
    earning_data = pd.read_csv(os.path.join(directory, file))

    earning_data.rename(columns={'earning date':'Earning Date'}, inplace=True)
    # Convert the date to datetime and check the conversion

    earning_data['Earning Date'] = pd.to_datetime(earning_data['Earning Date'], utc=True)
    #lable the earning date as 1
    earning_data['Earning'] = 1
    # Remove the time from the date
    earning_data['Earning Date'] = earning_data['Earning Date'].dt.date
    # just inc one day to get the next day for the 'Date'

    #if the earning date is not the same day as the earning collection date, add one day to the earning date
    if(earning_collection_date_dict[stock_name] == 1):
        earning_data['Date'] = earning_data['Earning Date'] + pd.DateOffset(1)
    else:
        earning_data['Date'] = earning_data['Earning Date']
    # Set the 'Date' column as the index
    earning_data.set_index('Date', inplace=True)

    stock = ab.StockData(stock_name)
    stock.get_data_from_yfinance(stock_name, sd, ed)

    # Merge the earning data with the stock data
    stock.data = pd.merge(stock.data, earning_data, how='left', left_index=True, right_index=True)
    #plot the stock data and lable the earning date
    #stock.data.plot(y=['Close'], figsize=(16,1.5), sharey=True, color=['blue'],linewidth=0.7 ,title=stock_name)

    #is it a gap up day？
    stock.data['Gap Up'] = stock.data['Low'] > stock.data['High'].shift(1)
    stock.data['Price Change%'] = stock.data['Close'].pct_change(periods=1)
    stock.data['Gap%[(Low-Last Day High)/(Last Day High)]'] = (stock.data['Low'] - stock.data['High'].shift(1))/stock.data['High'].shift(1)
    stock.data['Spread%[(High - Low)/Low]'] = (stock.data['High'] - stock.data['Low'])/stock.data['Low']

    #stock.get_sma('Close', 5, 'SMA5')
    stock.get_sma('Close', 10, 'SMA10')
    stock.get_sma('Close', 21, 'SMA21')
    stock.get_sma('Close', 50, 'SMA50')
    stock.get_sma('Close', 150, 'SMA150')
    stock.get_sma('Close', 200, 'SMA200')
    stock.get_sma('Volume', 50, 'VOL50MA')
    stock.get_close_range(high='High', low='Low', close='Close', range='Close Range')

    #stock.get_to_percentage('Close', 'SMA5', 'TO_SMA5')
    stock.get_to_percentage('Close', 'SMA10', 'TO_SMA10')
    stock.get_to_percentage('Close', 'SMA21', 'TO_SMA21')
    stock.get_to_percentage('Close', 'SMA50', 'TO_SMA50')
    stock.get_to_percentage('Close', 'SMA150', 'TO_SMA150')
    stock.get_to_percentage('Close', 'SMA200', 'TO_SMA200')
    stock.get_to_percentage('Volume', 'VOL50MA', 'TO_VOL50MA')
    #get the stock gain percentage after earning date in 1w 2w 4w 8w
    stock.data['Performanc 1W'] = -stock.data['Close'].pct_change(periods=-5)
    stock.data['Performanc 2W'] = -stock.data['Close'].pct_change(periods=-10)
    stock.data['Performanc 4W'] = -stock.data['Close'].pct_change(periods=-20)
    stock.data['Performanc 5W'] = -stock.data['Close'].pct_change(periods=-25)
    stock.data['Performanc 8W'] = -stock.data['Close'].pct_change(periods=-40)
    stock.data['Performanc 13W'] = -stock.data['Close'].pct_change(periods=-(13*5))

    #backfill After_Earning column with 1 after earning date==1
    stock.data['After Earning'] = stock.data['Earning'].fillna(method='ffill', limit=13*5)
    #'Earning_Low' populate the earning date low price to the next 40 days

    #check if the stock price is higher than earning date low
    #stock.data['Break_Low'] = stock.data['Close'] < stock.data['Earning_Low']


    days_counter = 0
    for index, row in stock.data.iterrows():
        # Check if Earning equals 1
        if row['Earning'] == 1:
            # Record the Low value and reset the counter
            low_value_on_earning = row['Low']
            days_counter = 40

        # If we are in the 40-day period after an Earning == 1 event
        if days_counter > 0:
            stock.data.at[index, 'Earning Low'] = low_value_on_earning
            # Decrease the counter
            days_counter -= 1


    stock.data['Undercut Earning Low'] = stock.data['Close'] < stock.data['Earning Low']
    #under cut low 5%
    stock.data['Undercut Earning Close 5%'] = stock.data['Close'] < stock.data['Earning Low'] * 0.95
    #8% loss from gap up closing price
    stock.data['Undercut Earning Low 8%'] = stock.data['Close'] < stock.data['Earning Low'] * 0.92


    #add up the undercut earning column then convert to boolean
    stock.data['Undercut Earning 1W'] = stock.data['Undercut Earning Low'].shift(-5).rolling(5).sum()
    stock.data['Undercut Earning 2W'] = stock.data['Undercut Earning Low'].shift(-10).rolling(10).sum()
    stock.data['Undercut Earning 4W'] = stock.data['Undercut Earning Low'].shift(-20).rolling(20).sum()
    stock.data['Undercut Earning 8W'] = stock.data['Undercut Earning Low'].shift(-40).rolling(40).sum()

    stock.data['Undercut Earning Close 5% 1W'] = stock.data['Undercut Earning Close 5%'].shift(-5).rolling(5).sum()
    stock.data['Undercut Earning Close 5% 2W'] = stock.data['Undercut Earning Close 5%'].shift(-10).rolling(10).sum()
    stock.data['Undercut Earning Close 5% 4W'] = stock.data['Undercut Earning Close 5%'].shift(-20).rolling(20).sum()
    stock.data['Undercut Earning Close 5% 8W'] = stock.data['Undercut Earning Close 5%'].shift(-40).rolling(40).sum()

    stock.data['Undercut Earning Low 8% 1W'] = stock.data['Undercut Earning Low 8%'].shift(-5).rolling(5).sum()
    stock.data['Undercut Earning Low 8% 2W'] = stock.data['Undercut Earning Low 8%'].shift(-10).rolling(10).sum()
    stock.data['Undercut Earning Low 8% 4W'] = stock.data['Undercut Earning Low 8%'].shift(-20).rolling(20).sum()
    stock.data['Undercut Earning Low 8% 8W'] = stock.data['Undercut Earning Low 8%'].shift(-40).rolling(40).sum()
    #stock.data['EPS%'] = stock.data['EPS'].pct_change(periods=1)
    #stock.data['PRICE%']
    #stock.data['VOL_TO50MA']
    stock.data['SUCCESS GAP UP'] = ~(stock.data['Undercut Earning 8W'] > 0) & ~(stock.data['Undercut Earning Close 5% 8W'] > 0)
    #drop Open High Low Close Volume Close Earning 'After Earning' 'Undercut Earning' SMA10 SMA21 SMA50 SMA150 SMA200 VOL50MA
    stock.data.drop(columns=['Adj Close', 'Volume', 'After Earning', 'Earning Low'], inplace=True)
    stock.data.drop(columns=['SMA10', 'SMA21', 'SMA50', 'SMA150', 'SMA200', 'VOL50MA', 'Undercut Earning Low', 'Undercut Earning Close 5%', 'Undercut Earning Low 8%',], inplace=True)

    stock.data = stock.data[(stock.data['Earning'] == 1)]
    #stock.data = stock.data[(stock.data['Earning'] == 1) | (stock.data['After Earning'] == 1)]

    stock.data.drop(columns=['Earning'], inplace=True)
    stock.data['ticker'] = stock_name
    #only keep 3 digits after decimal point
    stock.data = stock.data.round(3)
    stock.data['Close Range'] = stock.data['Close Range'].round(2)
    stock.data.to_csv(output+stock_name+'.csv', index=True)
    #store stock.data to all_data
    all_data = pd.concat([all_data, stock.data], axis=0)


all_data.to_csv(output + 'All_Data.csv', index=True)







{'AAPL': 1, 'ADBE': 1, 'ALGN': 1, 'ANET': 1, 'ANF': 0, 'AXON': 0, 'BIDU': 1, 'BKNG': 1, 'BLDR': 0, 'CELH': 1, 'CF': 1, 'DECK': 1, 'DELL': 1, 'DPZ': 0, 'DUOL': 1, 'EBAY': 1, 'ELF': 1, 'ENPH': 1, 'EXAS': 0, 'FIVN': 1, 'FOSL': 0, 'FSLR': 1, 'LNG': 0, 'MDB': 1, 'META': 1, 'MOS': 1, 'MSFT': 1, 'NFLX': 1, 'NIO': 0, 'NOW': 1, 'NTES': 0, 'NVDA': 1, 'PDD': 1, 'PLTR': 1, 'REGN': 0, 'SE': 0, 'SHOP': 0, 'SMCI': 1, 'SQ': 1, 'SWN': 0, 'TAL': 1, 'TEAM': 1, 'TSLA': 1, 'UBER': 0, 'ULTA': 1, 'VRT': 0, 'WB': 1, 'ZM': 1}
PLTR
[*********************100%%**********************]  1 of 1 completed
             Open    High     Low  Close  Weekday ticker Earning Date  \
Date                                                                    
2022-02-18  11.92  12.010  10.570  11.02        4   PLTR   2022-02-17   
2022-05-10   7.56   7.703   6.930   7.29        1   PLTR   2022-05-09   
2022-08-09   9.54   9.570   9.010   9.25        1   PLTR   2022-08-08   
2022-11-08   7.08   7.420   6.910   7.21        1   PL

/tmp/ipykernel_539944/1550245757.py:103: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock.data['After Earning'] = stock.data['Earning'].fillna(method='ffill', limit=13*5)


[*********************100%%**********************]  1 of 1 completed
              Open    High     Low   Close  Weekday ticker Earning Date  \
Date                                                                      
2022-01-26  307.99  308.50  293.03  296.71        2   MSFT   2022-01-25   
2022-04-27  282.10  290.97  279.16  283.22        2   MSFT   2022-04-26   
2022-07-27  261.16  270.05  258.85  268.74        2   MSFT   2022-07-26   
2022-10-26  231.17  238.30  230.06  231.32        2   MSFT   2022-10-25   
2023-01-25  234.48  243.30  230.90  240.61        2   MSFT   2023-01-24   
2023-04-26  296.70  299.57  292.73  295.37        2   MSFT   2023-04-25   
2023-07-26  341.44  344.67  333.11  337.77        2   MSFT   2023-07-25   
2023-10-25  345.02  346.20  337.62  340.67        2   MSFT   2023-10-24   

            estimated eps   reported eps  earning surprise  ...  \
Date                                                        ...   
2022-01-26            2.31           2.48     

/tmp/ipykernel_539944/1550245757.py:103: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock.data['After Earning'] = stock.data['Earning'].fillna(method='ffill', limit=13*5)


[*********************100%%**********************]  1 of 1 completed
              Open    High     Low   Close  Weekday ticker Earning Date  \
Date                                                                      
2020-02-07   27.41   28.74   24.50   27.55        4   SMCI   2020-02-06   
2020-05-08   23.50   26.89   23.01   25.42        4   SMCI   2020-05-07   
2020-08-12   27.84   28.45   25.64   26.01        2   SMCI   2020-08-11   
2020-11-04   26.87   28.88   26.22   28.38        2   SMCI   2020-11-03   
2021-02-03   32.50   34.74   32.50   33.03        2   SMCI   2021-02-02   
2021-05-05   35.59   36.96   35.20   36.26        2   SMCI   2021-05-04   
2021-08-11   39.50   39.63   38.13   38.82        2   SMCI   2021-08-10   
2021-11-03   41.41   44.00   40.26   41.87        2   SMCI   2021-11-02   
2022-02-02   43.52   44.01   40.94   43.81        2   SMCI   2022-02-01   
2022-05-04   48.32   56.96   48.25   56.72        2   SMCI   2022-05-03   
2022-08-10   63.76   68.80   63

...

/tmp/ipykernel_539944/1550245757.py:103: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock.data['After Earning'] = stock.data['Earning'].fillna(method='ffill', limit=13*5)


[*********************100%%**********************]  1 of 1 completed
               Open     High      Low   Close  Weekday ticker Earning Date  \
Date                                                                         
2020-08-06   20.840   21.020   19.300   20.54        3    ELF   2020-08-05   
2020-11-05   18.950   20.059   18.545   19.43        3    ELF   2020-11-04   
2021-02-04   22.240   24.150   21.290   23.12        3    ELF   2021-02-03   
2021-05-27   28.060   29.450   27.360   28.52        3    ELF   2021-05-26   
2021-08-05   30.750   31.230   29.610   29.83        3    ELF   2021-08-04   
2021-11-04   30.000   32.100   29.570   30.21        3    ELF   2021-11-03   
2022-02-03   29.950   30.650   28.020   28.52        3    ELF   2022-02-02   
2022-05-26   22.970   24.770   22.970   24.63        3    ELF   2022-05-25   
2022-08-04   36.450   36.740   34.760   36.18        3    ELF   2022-08-03   
2022-11-03   45.310   48.440   44.280   48.09        3    ELF   2022-11-0

/tmp/ipykernel_539944/1550245757.py:103: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock.data['After Earning'] = stock.data['Earning'].fillna(method='ffill', limit=13*5)


[*********************100%%**********************]  1 of 1 completed
               Open     High      Low    Close  Weekday ticker Earning Date  \
Date                                                                          
2008-02-14   27.400   28.044   25.965   26.450        3   BIDU   2008-02-13   
2008-04-25   35.806   36.706   35.248   36.369        4   BIDU   2008-04-24   
2008-07-24   32.600   34.100   32.140   33.508        3   BIDU   2008-07-23   
2008-10-23   23.983   24.033   20.600   21.254        3   BIDU   2008-10-22   
2009-02-19   13.462   13.900   13.183   13.257        3   BIDU   2009-02-18   
2009-04-28   22.955   23.662   21.735   22.119        1   BIDU   2009-04-27   
2009-07-24   33.990   36.330   33.808   35.810        4   BIDU   2009-07-23   
2009-10-27   35.571   38.820   35.303   38.366        1   BIDU   2009-10-26   
2010-02-10   47.188   48.294   46.501   48.213        2   BIDU   2010-02-09   
2010-04-29   70.796   71.800   70.100   70.987        3   BIDU

/tmp/ipykernel_539944/1550245757.py:103: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock.data['After Earning'] = stock.data['Earning'].fillna(method='ffill', limit=13*5)
